In [1]:
import firebase_admin
from pathlib import Path
from firebase_admin import credentials
from firebase_admin import firestore
import warnings
import os
warnings.filterwarnings('ignore')

In [2]:
# Use the path to your service account key file
PARENT_DIR = Path('__file__').resolve().parent
print(PARENT_DIR)


/Users/sunilthapa/Desktop/Project/geolocation-app/xcor


In [3]:
service_account_dir = '/Users/sunilthapa/Desktop/Project/'


In [4]:
cred = credentials.Certificate(os.path.join(service_account_dir,"serviceAccountKey.json"))

# Initialize the app with your project's credentials
# You can also specify the database URL if it's not the default
firebase_admin.initialize_app(cred, {
    'projectId': 'geolocation-app-b549e',
})

# Now you can use the Firebase services
# For example, to access Firestore:
db = firestore.client()

In [5]:
def get_all_sessions():
    """Retrieve all session data from Firestore"""
    if not db:
        print("Firebase not initialized. Please check your configuration.")
        return None
    
    try:
        # Get all documents from the 'locations' collection
        locations_ref = db.collection('sessions')
        docs = locations_ref.stream()
        
        locations_data = []
        for doc in docs:
            data = doc.to_dict()
            data['document_id'] = doc.id  # Add document ID for reference
            locations_data.append(data)
        
        print(f"Retrieved {len(locations_data)} location records")
        return locations_data
    except Exception as e:
        print(f"Error retrieving data: {e}")
        return None

In [5]:
import math

def haversine_distance(lat1, lon1, lat2, lon2, radius=6371):
    """
    Calculates the Haversine distance between two points on Earth.

    Args:
        lat1 (float): Latitude of the first point in degrees.
        lon1 (float): Longitude of the first point in degrees.
        lat2 (float): Latitude of the second point in degrees.
        lon2 (float): Longitude of the second point in degrees.
        radius (float): Radius of the Earth in kilometers (default is 6371 km).

    Returns:
        float: The distance between the two points in kilometers.
    """
    # Convert latitudes and longitudes from degrees to radians
    lat1_rad = math.radians(lat1)
    lon1_rad = math.radians(lon1)
    lat2_rad = math.radians(lat2)
    lon2_rad = math.radians(lon2)

    # Calculate the difference in latitudes and longitudes
    dlat = lat2_rad - lat1_rad
    dlon = lon2_rad - lon1_rad

    # Haversine formula
    a = math.sin(dlat / 2)**2 + math.cos(lat1_rad) * math.cos(lat2_rad) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = radius * c * 1000
    return distance



In [6]:
sessions_data = get_all_sessions()

if sessions_data:
    print(f"\nSuccessfully connected! Found {len(sessions_data)} location records.")
    print("\nSample data structure:")
    if sessions_data:
        sample = sessions_data[0]
        for key, value in sample.items():
            print(f"  {key}: {value}")
else:
    print("Failed to retrieve data. Please check your Firebase configuration.")


E0000 00:00:1763408467.149773 55907699 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


Retrieved 16 location records

Successfully connected! Found 16 location records.

Sample data structure:
  participants: [{'audioUrl': 'https://firebasestorage.googleapis.com/v0/b/geolocation-app-b549e.firebasestorage.app/o/audio%2Fgeneral_a2_2025-11-17_10%3A10.webm?alt=media&token=21210740-e649-4c8d-ae93-a03f0f2a3853', 'deviceName': 'Windows PC (Chrome)', 'joinedAt': '2025-11-17T16:10:00.762Z', 'username': 'a2', 'location': {'timestamp': '11/17/2025, 10:07:37 AM', 'longitude': -90.4688, 'accuracy': 48272, 'latitude': 30.5155}}]
  createdAt: 2025-11-17 16:10:00.883000+00:00
  events: [{'timestamp': '2025-11-17T16:10:00.836Z', 'by': 'a2', 'type': 'saved'}, {'by': 'a2', 'audioUrl': 'https://firebasestorage.googleapis.com/v0/b/geolocation-app-b549e.firebasestorage.app/o/audio%2Fgeneral_a2_2025-11-17_10%3A10.webm?alt=media&token=21210740-e649-4c8d-ae93-a03f0f2a3853', 'timestamp': '2025-11-17T16:10:00.836Z', 'type': 'recording_uploaded'}]
  name: local_locations
  document_id: local_a2


In [9]:
# lat_s, lon_s = locations_data[0]['latitude'], locations_data[0]['longitude']
# lat_e, lon_e = locations_data[2]['latitude'], locations_data[2]['longitude']

# distance = haversine_distance(lat_s, lon_s, lat_e, lon_e)
# print(f"Distance between {locations_data[0]['username']} and {locations_data[2]['username']}: {distance:.2f} m")


In [7]:
sessions_data[0]['participants'][0]['audioUrl']

'https://firebasestorage.googleapis.com/v0/b/geolocation-app-b549e.firebasestorage.app/o/audio%2Fgeneral_a2_2025-11-17_10%3A10.webm?alt=media&token=21210740-e649-4c8d-ae93-a03f0f2a3853'

In [8]:
import librosa
import io

ModuleNotFoundError: No module named 'librosa'

In [12]:
'''
import requests
par_session = sessions_data[2]['participants'] 
for i in range(len(par_session)):
    try:
        print(i)
        audio = par_session[i]['audioUrl']
        response = requests.get(audio)
        print('----')
        audio_data = response.content
        print(audio_data)
        audio_io = io.BytesIO(audio_data)
        print('===')
        y, sr = librosa.load(audio_io, sr=None)
        print('+++')
        print(y, sr)
        break
        # with open(f'audio_{par_session[i]["username"]}.webm', 'wb') as f:
        #     f.write(response.content)
    except Exception as e:
        print(f"Error downloading audio for {par_session[i]['username']}: {e}")
'''


'\nimport requests\npar_session = sessions_data[2][\'participants\'] \nfor i in range(len(par_session)):\n    try:\n        print(i)\n        audio = par_session[i][\'audioUrl\']\n        response = requests.get(audio)\n        print(\'----\')\n        audio_data = response.content\n        print(audio_data)\n        audio_io = io.BytesIO(audio_data)\n        print(\'===\')\n        y, sr = librosa.load(audio_io, sr=None)\n        print(\'+++\')\n        print(y, sr)\n        break\n        # with open(f\'audio_{par_session[i]["username"]}.webm\', \'wb\') as f:\n        #     f.write(response.content)\n    except Exception as e:\n        print(f"Error downloading audio for {par_session[i][\'username\']}: {e}")\n'

In [2]:
import os
from pathlib import Path

import librosa
import numpy as np
import matplotlib.pyplot as plt

from scipy.io import wavfile
from scipy.fft import fft, fftfreq

In [15]:
def plot_audio_analysis(audio_path):
    # Load audio file
    print('INsert')
    y, sr = librosa.load(audio_path, sr=None)  # y = waveform, sr = sample rate
    print('Read data')
    duration = len(y) / sr
    time = np.linspace(0, duration, len(y))

    # FFT
    N = len(y)
    yf = fft(y)
    xf = fftfreq(N, 1 / sr)[:N // 2]
    spectrum = np.abs(yf[:N // 2])

    # Spectrogram
    S = librosa.stft(y)
    S_db = librosa.amplitude_to_db(np.abs(S), ref=np.max)

    # Plot
    fig, axs = plt.subplots(3, 1, figsize=(12, 10))

    # Waveform
    axs[0].plot(time, y, color='blue')
    axs[0].set_title('Waveform')
    axs[0].set_xlabel('Time [s]')
    axs[0].set_ylabel('Amplitude')

    # FFT Spectrum
    axs[1].plot(xf, spectrum, color='green')
    axs[1].set_title('FFT Spectrum')
    axs[1].set_xlabel('Frequency [Hz]')
    axs[1].set_ylabel('Magnitude')
    axs[1].set_xlim(0, sr / 2)  # Limit to Nyquist

    # Spectrogram
    img = librosa.display.specshow(S_db, sr=sr, x_axis='time', y_axis='log', ax=axs[2], cmap='gray_r')
    axs[2].set_title('Spectrogram (dB)')
    #fig.colorbar(img, ax=axs[2], format='%+2.0f dB')

    plt.tight_layout()
    plt.show()

In [19]:
import requests

In [26]:
def download_audio(participant_session_data):
    audio = participant_session_data['audioUrl']
    response = requests.get(audio)
    audio_name = f'audio_{participant_session_data["username"]}.wav'
    print(audio_name)
    with open(audio_name, 'wb') as f:
        f.write(response.content)

In [27]:
for sess in sessions_data:
    if sess['name'].lower() == 'mead1':
        for part in sess['participants']:
            download_audio(part)
        break

audio_mead1.wav
audio_a2.wav


In [17]:
import requests
par_session = sessions_data[2]['participants'] 
for i in range(len(par_session)):
    try:
        audio = par_session[i]['audioUrl']
        response = requests.get(audio)
        audio_name = f'audio_{par_session[i]["username"]}.wav'
        with open(audio_name, 'wb') as f:
            f.write(response.content)

        plot_audio_analysis(audio_name)
        
    except Exception as e:
        print(f"Error downloading audio for {par_session[i]['username']}: {e}")


INsert
Error downloading audio for a1: 
INsert
Error downloading audio for e1: 
INsert
Error downloading audio for s1: 


# 